<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark6_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [ ]:
import os
from os.path import exists, join, basename, splitext
import json
import numpy as np

#Aufbau Json File genau anschauen

json_path = "/content.gdrive/My Drive//Train2/data.json"
#json_path = "/content/data.json"
# dictionary to store mapping, labels, and Keypoints
data = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/My Drive/Train2"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt

    folder_dir = Path_Pic + '/exp'
    %cd /content/openpose/build/examples/tutorial_api_python
    !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true

    with open(Path_Pic + '/exp/keypoints.json', "r") as fp:
        keypoints_json = json.load(fp)

    data["keypoints"].append(keypoints_json)

    #with open(Path_Pic + '/exp/keypoints.txt', "r") as keypointfile:
        #keypoints = keypointfile.read()
    #data["keypoints"].append(keypoints)


with open(json_path, "w") as fp:
  json.dump(data, fp, indent=4)


In [57]:

#test load data
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

with open('/content.gdrive/MyDrive/Train2/data.json', "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
X = np.array(data["keypoints"])
y = np.array(data["labels"])

print(y)
print(type(y))
print(type(X))
print(X)


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


[['NoTelemark' 'Telemark']]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0.61342597 0.30372337 0.23925512 ... 0.52524942 0.71154225 0.54309702]
 [0.4108656  0.33063355 0.24723598 ... 0.4081504  0.68497944 0.33570752]
 [0.70509571 0.25489008 0.19814925 ... 0.59326494 0.7493878  0.5951854 ]]


In [11]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "/content.gdrive/MyDrive/Train2/data (2).json"

def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["keypoints"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y


if __name__ == "__main__":

    # load data
    X, y = load_data(DATA_PATH)


    

Data succesfully loaded!
[['NoTelemark' 'Telemark']]
['[[0.6134259700775146, 0.30372336506843567, 0.2392551153898239, 0.534103274345398, 0.3259412944316864, 0.6454670429229736, 0.5089718699455261, 0.3232909142971039, 0.6135991811752319, 0.4336528480052948, 0.3957844376564026, 0.6436108946800232, 0.34160640835762024, 0.46017736196517944, 0.6659100651741028, 0.5716989636421204, 0.32880455255508423, 0.5051397681236267, 0.5423445701599121, 0.3986267149448395, 0.07582145184278488, 0.0, 0.0, 0.0, 0.46296390891075134, 0.4488585591316223, 0.647128701210022, 0.42943692207336426, 0.44893380999565125, 0.6155615448951721, 0.5381457805633545, 0.5802322626113892, 0.6620995402336121, 0.521531343460083, 0.7253342866897583, 0.6022000908851624, 0.496476948261261, 0.4433518946170807, 0.6237298250198364, 0.5799645781517029, 0.5494794845581055, 0.604781448841095, 0.5466185212135315, 0.7003284096717834, 0.37619441747665405, 0.6050760746002197, 0.2869557738304138, 0.2603975534439087, 0.6217179894447327, 0.29